# Group Member:

* Xiao Jun Qiu
* Tao Shan
* E Ching Kho

# Preprocessing

We filled in missing values and convert category variables to dummy variables or convert to other compesite variables that make more sense.

Data preprocessing included relabeling original categorical labels to precise age groups and applying binary flags for categorical labels with high prevalence. Additionally, regular expression patterns were utilized to distill information from textual data. Variables such as RacingSubType, SexRestriction, StartType, and Surface were converted into new indicators that reflect particular subtypes, restrictions, or conditions. Below is the list of variables that have been modified:

* AgeRestriction
* Barrier
* ClassRestriction
* CourseIndicator
* DamID
* FoalingDate
* HandicapType
* RaceGroup
* RaceStartTime
* RacingSubType
* SexRestriction
* StartType
* Surface

#Constructing the Win Probability Metric

In quantifying the likelihood of winning, we adopted the distribution of prize money as a proxy for the true win probability. This innovative approach guarantees that the aggregated probability for each raceID is sum to 1.

We devised a custom normalization function that adeptly adjusts the predicted win probabilities for the trained model, ensuring their collective sum per raceID achieves the desired total of 1.

# Modeling

The modeling approach incorporates techniques such as correlation-based feature selection and Min-Max scaling to optimize data compatibility with the algorithms. The implemented methods are:

* Decision Tree
* Random Forest
* Gradient Boosting Tree
* XGBoost
* Linear Regression
* Logistic Regression
* Lasso Regression

# Conclusive Findings

Upon meticulous evaluation, we concluded that the XGBoost model exhibited superior performance relative to its counterparts by MAE metrics. Consequently, we have nominated this model for our official submission, given its exceptional predictive prowess.

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV, LinearRegression, LogisticRegressionCV

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DIR_PATH = "/content/drive/MyDrive/CANSSI Competition/"

Mounted at /content/drive


In [ ]:
# Read Parquet file
df = pd.read_parquet(DIR_PATH+'trots_2013-2022.parquet', engine='pyarrow')
df.head()

,AgeRestriction,Barrier,BeatenMargin,ClassRestriction,CourseIndicator,DamID,Disqualified,Distance,FinishPosition,FoalingCountry,...,StartType,StartingLine,Surface,TrackID,TrainerID,NoFrontCover,PositionInRunning,WideOffRail,WeightCarried,WetnessScale
0,6yo,5,1.55,NW$101 CD,,1491946,False,2150.0,2,FR,...,M,1,S,951,38190,-9,-9,-9,0.0,3
1,6yo,6,3.55,NW$101 CD,,1509392,False,2150.0,4,FR,...,M,1,S,951,38432,-9,-9,-9,0.0,3
2,6yo,7,5.55,NW$101 CD,,1507967,False,2150.0,6,FR,...,M,1,S,951,37826,-9,-9,-9,0.0,3
3,6yo,8,999.00,NW$101 CD,,1508536,False,2150.0,BS,FR,...,M,1,S,951,38293,-9,-9,-9,0.0,3
4,6yo,9,999.00,NW$101 CD,,1514055,False,2150.0,BS,FR,...,M,2,S,951,38945,-9,-9,-9,0.0,3


In [ ]:
# use a sample to test preprocessing
#df = df.sample(1000, random_state=42)
# df = df.head(1000)

# Data preprocessing

Tao Shan is taking care of variables:
AgeRestriction, Barrier, BeatenMargin, ClassRestriction, CourseIndicator, DamID, Disqualified, Distance, FinishPosition, FoalingCountry, FoalingDate, FrontShoes, Gender, GoingAbbrev, GoingID

There are 7 variables he modified:
- AgeRestriction (9 new variables)
- Barrier (1 new variables)
- ClassRestriction (2 new variables)
- CourseIndicator (1 new variables)
- DamID (2 new variables)
- FinishPosition (2 new variables)
- FoalingDate (3 new variables)

There are 5 variables that he removed:
- AgeRestriction
- ClassRestriction
- DamID
- FinishPosition
- FoalingDate


In [ ]:
# Prepare new columns
for i in range(3, 11):
    df[f'AgeRestriction_{i}'] = 0

# Regex patterns
single_age_pattern = re.compile(r'(\d+)yo\+?')
range_age_pattern = re.compile(r'(\d+)-(\d+)yo')
multi_age_pattern = re.compile(r'(\d+)&(\d+)yo')

def update_row(row):
    age_str = str(row['AgeRestriction']).strip()

    single_age_match = single_age_pattern.match(age_str)
    if single_age_match:
        age = int(single_age_match.group(1))
        valid_cols = [f'AgeRestriction_{i}' for i in range(age, 11) if f'AgeRestriction_{i}' in df.columns]
        row[valid_cols] = 1

    range_age_match = range_age_pattern.match(age_str)
    if range_age_match:
        start, end = map(int, range_age_match.groups())
        valid_cols = [f'AgeRestriction_{i}' for i in range(start, end + 1) if f'AgeRestriction_{i}' in df.columns]
        row[valid_cols] = 1

    multi_age_match = multi_age_pattern.match(age_str)
    if multi_age_match:
        ages = list(map(int, multi_age_match.groups()))
        valid_cols = [f'AgeRestriction_{age}' for age in ages if f'AgeRestriction_{age}' in df.columns]
        row[valid_cols] = 1

    return row

df = df.progress_apply(update_row, axis=1)

print(df[["AgeRestriction"]+[f'AgeRestriction_{i}' for i in range(3, 11)]])

  0%|          | 0/1200412 [00:00<?, ?it/s]

        AgeRestriction  AgeRestriction_3  AgeRestriction_4  AgeRestriction_5  \
0                  6yo                 0                 0                 0   
1                  6yo                 0                 0                 0   
2                  6yo                 0                 0                 0   
3                  6yo                 0                 0                 0   
4                  6yo                 0                 0                 0   
...                ...               ...               ...               ...   
1200407          8&9yo                 0                 0                 0   
1200408          8&9yo                 0                 0                 0   
1200409          8&9yo                 0                 0                 0   
1200410            6yo                 0                 0                 0   
1200411            6yo                 0                 0                 0   

         AgeRestriction_6  AgeRestricti

In [ ]:
# # Directly update the DataFrame for new AgeRestriction columns
# for index, row in tqdm(df.iterrows()):
#     age_str = str(row['AgeRestriction']).strip()

#     # Single age (e.g., "3yo" or "3yo+")
#     if age_str.endswith('yo') or age_str.endswith('yo+'):
#         try:
#             age = int(age_str.split('yo')[0])
#             for i in range(age, 11):
#                 df.at[index, f'AgeRestriction_{i}'] = 1
#         except ValueError:
#             pass  # Handle age ranges and lists next

#     # Age range (e.g., "3-5yo")
#     if '-' in age_str:
#         age_range = list(map(int, age_str.split('yo')[0].split('-')))
#         for i in range(age_range[0], age_range[1] + 1):
#             df.at[index, f'AgeRestriction_{i}'] = 1

#     # Multiple ages (e.g., "7&8yo")
#     if '&' in age_str:
#         ages = list(map(int, age_str.split('yo')[0].split('&')))
#         for age in ages:
#             df.at[index, f'AgeRestriction_{age}'] = 1

# # Show the first few rows to confirm changes
# df[["AgeRestriction"]+[f'AgeRestriction_{i}' for i in range(3, 11)]].head()


In [ ]:
# flag if the value is 0 or not
df['Barrier_Flag'] = df['Barrier'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
# extract numbers and letters
import re

# Function to extract the numerical value after 'NW$'
def extract_nw_value(value):
    match = re.search(r'NW\$(\d+)', value)
    return int(match.group(1)) if match else 0

# Function to flag if a certain substring is present
def flag_substring(value, substring):
    return 1 if substring in value else 0

# Create new column with the numerical value after 'NW$'
df['NW_Value'] = df['ClassRestriction'].apply(lambda x: extract_nw_value(str(x)))

# Create new columns for specific substrings (example: 'CA')
df['Flag_CA'] = df['ClassRestriction'].apply(lambda x: flag_substring(str(x), 'CF'))

In [ ]:
df[['ClassRestriction', 'NW_Value', 'Flag_CA']].head()

,ClassRestriction,NW_Value,Flag_CA
0,NW$101 CD,101,0
1,NW$101 CD,101,0
2,NW$101 CD,101,0
3,NW$101 CD,101,0
4,NW$101 CD,101,0


In [ ]:
# flag if the value is 0 or not
df['CourseIndicator_Flag'] = df['CourseIndicator'].apply(lambda x: 0 if x == "" else 1)

In [ ]:
# Dam ID: frequency encoding
# Calculate the frequency of each unique value in the 'DamID' column
frequency_map = df['DamID'].value_counts().to_dict()

# Create a new column 'DamID_Frequency' that holds the frequency of each 'DamID'
df['DamID_Frequency'] = df['DamID'].map(frequency_map)

# Show sample data to confirm that the frequency encoding has been applied
df[['DamID', 'DamID_Frequency']].head(10)


,DamID,DamID_Frequency
0,1491946,54
1,1509392,70
2,1507967,26
3,1508536,81
4,1514055,47
5,1490328,97
6,1495060,108
7,1496640,89
8,1479721,23
9,1480787,81


In [ ]:
# flag if the value is 0 or not
df['DamID_Frequency_Flag'] = df['DamID_Frequency'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
df['Finish_Flag'] = df['FinishPosition'].str.contains(r'\d', regex=True).astype(int)
df['Non_Finish_Status'] = df.apply(lambda row: 'finished' if row['Finish_Flag'] == 1 else row['FinishPosition'], axis=1)
df[['Finish_Flag','Non_Finish_Status']]

,Finish_Flag,Non_Finish_Status
0,1,finished
1,1,finished
2,1,finished
3,0,BS
4,0,BS
...,...,...
1200407,0,BS
1200408,0,BS
1200409,0,BS
1200410,1,finished


In [ ]:
df['FoalingDate_Year'] = df['FoalingDate'].dt.year
df['FoalingDate_Month'] = df['FoalingDate'].dt.month
df['FoalingDate_Day_of_Week'] = df['FoalingDate'].dt.day_name()

In [ ]:
win_probability_map = {
    '1  ': 0.45,
    '2  ': 0.25,
    '3  ': 0.14,
    '4  ': 0.08,
    '5  ': 0.05,
    '6  ': 0.02,
    '7  ': 0.01
}

# Create the new column using map, and for values >= 8, set as 0
df['win probability'] = df['FinishPosition'].map(win_probability_map).fillna(0)

In [ ]:
df.drop(['DamID','ClassRestriction','AgeRestriction','FinishPosition','FoalingDate'], axis = 1, inplace = True)

In [ ]:
# df.head()
df.columns

Index(['Barrier', 'BeatenMargin', 'CourseIndicator', 'Disqualified',
       'Distance', 'FoalingCountry', 'FrontShoes', 'Gender', 'GoingAbbrev',
       'GoingID', 'HandicapDistance', 'HandicapType', 'HindShoes', 'HorseAge',
       'HorseID', 'JockeyID', 'PIRPosition', 'PriceSP', 'Prizemoney',
       'RaceGroup', 'RaceID', 'RaceOverallTime', 'RacePrizemoney',
       'RaceStartTime', 'RacingSubType', 'Saddlecloth', 'SexRestriction',
       'SireID', 'StartType', 'StartingLine', 'Surface', 'TrackID',
       'TrainerID', 'NoFrontCover', 'PositionInRunning', 'WideOffRail',
       'WeightCarried', 'WetnessScale', 'AgeRestriction_3', 'AgeRestriction_4',
       'AgeRestriction_5', 'AgeRestriction_6', 'AgeRestriction_7',
       'AgeRestriction_8', 'AgeRestriction_9', 'AgeRestriction_10',
       'Barrier_Flag', 'NW_Value', 'Flag_CA', 'CourseIndicator_Flag',
       'DamID_Frequency', 'DamID_Frequency_Flag', 'Finish_Flag',
       'Non_Finish_Status', 'FoalingDate_Year', 'FoalingDate_Month',
      

E Ching Kho is taking care of variables:
HandicapDistance, HandicapType, HindShoes, HorseAge, HorseID, JockeyID, PIRPosition, PriceSP, Prizemoney, RaceGroup, RaceID, RaceOverallTime, RacePrizemoney, RaceStartTime

There are 3 variables he modified:
- HandicapType (3 new variable)
- RaceGroup (3 new variable)
- RaceStartTime (2 new variable)

There are 4 variables that he removed:
- HandicapType
- RaceGroup
- RaceStartTime
- RaceStartTime_time (created from RaceStartTime)

Thus, the number of columns in the updated dataset is
58 - 4 + (3 + 3 + 2) = 62

In [ ]:
# Set up conditions for RaceStartTime and create and a new column RaceStartTime_time
df["RaceStartTime_time"] = df['RaceStartTime'].dt.hour
conditions = [
    (df['RaceStartTime_time'] < 12),
    (df['RaceStartTime_time'] >= 12) & (df['RaceStartTime_time'] < 18),
    (df['RaceStartTime_time'] >= 18)
]
values = ['Morning', 'Afternoon', 'Evening']

df['RaceStartTime_time'] = np.select(conditions, values, default='')

In [ ]:
print("Number of unique values:",df["RaceStartTime_time"].nunique())
print("Sample values:",df["RaceStartTime_time"].unique())
df[['RaceStartTime', 'RaceStartTime_time']].head()

Number of unique values: 3
Sample values: ['Afternoon' 'Morning' 'Evening']


,RaceStartTime,RaceStartTime_time
0,2015-04-18 14:01:00,Afternoon
1,2015-04-18 14:01:00,Afternoon
2,2015-04-18 14:01:00,Afternoon
3,2015-04-18 14:01:00,Afternoon
4,2015-04-18 14:01:00,Afternoon


In [ ]:
# Create dummy variable columns for HandicapType, RaceGroup, and RaceStartTime_time
df_handtype = pd.get_dummies(df["HandicapType"], prefix="HandicapType", drop_first=True) # '' is the reference category
print("Number of unique values:",df["HandicapType"].nunique())
print("Sample values:",df["HandicapType"].unique())
df_handtype

Number of unique values: 4
Sample values: ['' 'Hcp' 'Cwt' 'SW']


,HandicapType_Cwt,HandicapType_Hcp,HandicapType_SW
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
1200407,0,0,0
1200408,0,0,0
1200409,0,0,0
1200410,0,0,0


In [ ]:
df_racegroup = pd.get_dummies(df["RaceGroup"], prefix="Race", drop_first=True) # ' ' is the reference category
print("Number of unique values:",df["RaceGroup"].nunique())
print("Sample values:",df["RaceGroup"].unique())
df_racegroup

Number of unique values: 4
Sample values: ['  ' 'G2' 'G3' 'G1']


,Race_G1,Race_G2,Race_G3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
1200407,0,0,0
1200408,0,0,0
1200409,0,0,0
1200410,0,0,0


In [ ]:
df_racestarttimegroup = pd.get_dummies(df["RaceStartTime_time"], prefix="Race", drop_first=True) # Afternoon is the reference category
df_racestarttimegroup

,Race_Evening,Race_Morning
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
1200407,0,0
1200408,0,0
1200409,0,0
1200410,0,0


In [ ]:
# Add the new columns
df = pd.concat([df,df_handtype,df_racegroup, df_racestarttimegroup], axis=1)
time = df['RaceStartTime']
race_id = df['RaceID']
# Remove unnecessary columns
df.drop(['HandicapType','RaceGroup','RaceStartTime_time'], axis = 1, inplace = True)

In [ ]:
print(len(df.columns))
df.columns

63


Index(['Barrier', 'BeatenMargin', 'CourseIndicator', 'Disqualified',
       'Distance', 'FoalingCountry', 'FrontShoes', 'Gender', 'GoingAbbrev',
       'GoingID', 'HandicapDistance', 'HindShoes', 'HorseAge', 'HorseID',
       'JockeyID', 'PIRPosition', 'PriceSP', 'Prizemoney', 'RaceID',
       'RaceOverallTime', 'RacePrizemoney', 'RacingSubType', 'Saddlecloth',
       'SexRestriction', 'SireID', 'StartType', 'StartingLine', 'Surface',
       'TrackID', 'TrainerID', 'NoFrontCover', 'PositionInRunning',
       'WideOffRail', 'WeightCarried', 'WetnessScale', 'AgeRestriction_3',
       'AgeRestriction_4', 'AgeRestriction_5', 'AgeRestriction_6',
       'AgeRestriction_7', 'AgeRestriction_8', 'AgeRestriction_9',
       'AgeRestriction_10', 'Barrier_Flag', 'NW_Value', 'Flag_CA',
       'CourseIndicator_Flag', 'DamID_Frequency', 'DamID_Frequency_Flag',
       'Finish_Flag', 'Non_Finish_Status', 'FoalingDate_Year',
       'FoalingDate_Month', 'FoalingDate_Day_of_Week', 'win probability',
      

Sunny is taking care of variables:
RacingSubType, Saddlecloth, SexRestriction, SirelD, StartType, StartingLine, Surface, TrackID, TrainerID, NoFrontCover, PositionInRunning, WideOffRail, WeightCarried, WetnessScale

There are 4 variables she modified:
- RacingSubType (1 new variable)
- SexRestriction (3 new variables)
- StartType (1 new variable)
- Surface (2 new variables)

There are 4 variables that he removed:
- RacingSubType
- SexRestriction
- StartType
- Surface

Thus, the number of columns in the updated dataset is
62 - 4 + (1 + 3 + 1 + 2) = 65

In [ ]:
print("Number of unique values:",df["RacingSubType"].nunique())
print("Sample values:",df["RacingSubType"].unique())

df["RacingSubType_T"] = df['RacingSubType'] == 'T ' # 'T ' is True, 'TM' is False

print(df["RacingSubType_T"].head())
df

Number of unique values: 2
Sample values: ['T ' 'TM']
0    True
1    True
2    True
3    True
4    True
Name: RacingSubType_T, dtype: bool


,Barrier,BeatenMargin,CourseIndicator,Disqualified,Distance,FoalingCountry,FrontShoes,Gender,GoingAbbrev,GoingID,...,win probability,HandicapType_Cwt,HandicapType_Hcp,HandicapType_SW,Race_G1,Race_G2,Race_G3,Race_Evening,Race_Morning,RacingSubType_T
0,5,1.55,,False,2150.0,FR,0,F,G,4,...,0.25,0,0,0,0,0,0,0,0,True
1,6,3.55,,False,2150.0,FR,0,F,G,4,...,0.08,0,0,0,0,0,0,0,0,True
2,7,5.55,,False,2150.0,FR,0,F,G,4,...,0.02,0,0,0,0,0,0,0,0,True
3,8,999.00,,False,2150.0,FR,0,F,G,4,...,0.00,0,0,0,0,0,0,0,0,True
4,9,999.00,,False,2150.0,FR,0,F,G,4,...,0.00,0,0,0,0,0,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200407,0,999.00,G,False,2850.0,FR,0,F,G,4,...,0.00,0,0,0,0,0,0,0,0,True
1200408,0,999.00,G,False,2850.0,FR,0,F,G,4,...,0.00,0,0,0,0,0,0,0,0,True
1200409,0,999.00,G,False,2850.0,FR,1,M,G,4,...,0.00,0,0,0,0,0,0,0,0,True
1200410,0,0.00,G,False,2850.0,ITY,0,F,G,4,...,0.45,0,0,0,0,0,0,0,0,True


In [ ]:
print("Number of unique values:",df["SexRestriction"].nunique())
print("Sample values:",df["SexRestriction"].unique())

df_sexrestriction = pd.get_dummies(df["SexRestriction"], prefix="SexRestriction", drop_first=True) # '' is the reference category
df_sexrestriction

Number of unique values: 4
Sample values: ['M' '' 'F' 'C&G']


,SexRestriction_C&G,SexRestriction_F,SexRestriction_M
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
1200407,0,0,0
1200408,0,0,0
1200409,0,0,0
1200410,0,0,1


In [ ]:
print("Number of unique values:",df["StartType"].nunique())
print("Sample values:",df["StartType"].unique())

df["StartType_V"] = df["StartType"] == "V"

print(df["StartType_V"].head())
df

Number of unique values: 2
Sample values: ['M' 'V']
0    False
1    False
2    False
3    False
4    False
Name: StartType_V, dtype: bool


,Barrier,BeatenMargin,CourseIndicator,Disqualified,Distance,FoalingCountry,FrontShoes,Gender,GoingAbbrev,GoingID,...,HandicapType_Cwt,HandicapType_Hcp,HandicapType_SW,Race_G1,Race_G2,Race_G3,Race_Evening,Race_Morning,RacingSubType_T,StartType_V
0,5,1.55,,False,2150.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,False
1,6,3.55,,False,2150.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,False
2,7,5.55,,False,2150.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,False
3,8,999.00,,False,2150.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,False
4,9,999.00,,False,2150.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200407,0,999.00,G,False,2850.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,True
1200408,0,999.00,G,False,2850.0,FR,0,F,G,4,...,0,0,0,0,0,0,0,0,True,True
1200409,0,999.00,G,False,2850.0,FR,1,M,G,4,...,0,0,0,0,0,0,0,0,True,True
1200410,0,0.00,G,False,2850.0,ITY,0,F,G,4,...,0,0,0,0,0,0,0,0,True,True


In [ ]:
print("Number of unique values:",df["Surface"].nunique())
print("Sample values:",df["Surface"].unique())

df_surface = pd.get_dummies(df["Surface"], prefix="Surface", drop_first=True) # 'C' is the reference category
df_surface

Number of unique values: 3
Sample values: ['S' 'C' 'T']


,Surface_S,Surface_T
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
1200407,0,0
1200408,0,0
1200409,0,0
1200410,0,0


In [ ]:
# Add parts of the new columns to df
df = pd.concat([df,df_sexrestriction,df_surface], axis=1)

# Remove unnecessary columns
df.drop(['RacingSubType','SexRestriction','StartType','Surface'], axis = 1, inplace = True)

In [ ]:
print(len(df.columns))
df.columns

66


Index(['Barrier', 'BeatenMargin', 'CourseIndicator', 'Disqualified',
       'Distance', 'FoalingCountry', 'FrontShoes', 'Gender', 'GoingAbbrev',
       'GoingID', 'HandicapDistance', 'HindShoes', 'HorseAge', 'HorseID',
       'JockeyID', 'PIRPosition', 'PriceSP', 'Prizemoney', 'RaceID',
       'RaceOverallTime', 'RacePrizemoney', 'Saddlecloth', 'SireID',
       'StartingLine', 'TrackID', 'TrainerID', 'NoFrontCover',
       'PositionInRunning', 'WideOffRail', 'WeightCarried', 'WetnessScale',
       'AgeRestriction_3', 'AgeRestriction_4', 'AgeRestriction_5',
       'AgeRestriction_6', 'AgeRestriction_7', 'AgeRestriction_8',
       'AgeRestriction_9', 'AgeRestriction_10', 'Barrier_Flag', 'NW_Value',
       'Flag_CA', 'CourseIndicator_Flag', 'DamID_Frequency',
       'DamID_Frequency_Flag', 'Finish_Flag', 'Non_Finish_Status',
       'FoalingDate_Year', 'FoalingDate_Month', 'FoalingDate_Day_of_Week',
       'win probability', 'HandicapType_Cwt', 'HandicapType_Hcp',
       'HandicapType_SW',

In [ ]:
df.shape

(1200412, 66)

In [ ]:
# df.to_parquet('preprocessed_datatrots_2013-2022.parquet')

Additional Preprocessing (to fit with regression models)

In [ ]:
# drop performance columns
columns_to_drop = ['BeatenMargin', 'Disqualified', 'PIRPosition',
                   'Prizemoney', 'RaceOverallTime', 'PriceSP', 'NoFrontCover',
                   'PositionInRunning', 'WideOffRail', 'Finish_Flag', 'Non_Finish_Status',]

df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
df.head()

,Barrier,CourseIndicator,Distance,FoalingCountry,FrontShoes,Gender,GoingAbbrev,GoingID,HandicapDistance,HindShoes,...,Race_G3,Race_Evening,Race_Morning,RacingSubType_T,StartType_V,SexRestriction_C&G,SexRestriction_F,SexRestriction_M,Surface_S,Surface_T
0,5,,2150.0,FR,0,F,G,4,0.0,0,...,0,0,0,True,False,0,0,1,1,0
1,6,,2150.0,FR,0,F,G,4,0.0,0,...,0,0,0,True,False,0,0,1,1,0
2,7,,2150.0,FR,0,F,G,4,0.0,0,...,0,0,0,True,False,0,0,1,1,0
3,8,,2150.0,FR,0,F,G,4,0.0,0,...,0,0,0,True,False,0,0,1,1,0
4,9,,2150.0,FR,0,F,G,4,0.0,0,...,0,0,0,True,False,0,0,1,1,0


In [ ]:
df.columns

Index(['Barrier', 'CourseIndicator', 'Distance', 'FoalingCountry',
       'FrontShoes', 'Gender', 'GoingAbbrev', 'GoingID', 'HandicapDistance',
       'HindShoes', 'HorseAge', 'HorseID', 'JockeyID', 'RaceID',
       'RacePrizemoney', 'Saddlecloth', 'SireID', 'StartingLine', 'TrackID',
       'TrainerID', 'WeightCarried', 'WetnessScale', 'AgeRestriction_3',
       'AgeRestriction_4', 'AgeRestriction_5', 'AgeRestriction_6',
       'AgeRestriction_7', 'AgeRestriction_8', 'AgeRestriction_9',
       'AgeRestriction_10', 'Barrier_Flag', 'NW_Value', 'Flag_CA',
       'CourseIndicator_Flag', 'DamID_Frequency', 'DamID_Frequency_Flag',
       'FoalingDate_Year', 'FoalingDate_Month', 'FoalingDate_Day_of_Week',
       'win probability', 'HandicapType_Cwt', 'HandicapType_Hcp',
       'HandicapType_SW', 'Race_G1', 'Race_G2', 'Race_G3', 'Race_Evening',
       'Race_Morning', 'RacingSubType_T', 'StartType_V', 'SexRestriction_C&G',
       'SexRestriction_F', 'SexRestriction_M', 'Surface_S', 'Surface_T'

In [ ]:
for col in df.columns:
    print("column name:",col)
    print("Number of unique values:",df[col].nunique())
    print("Sample values:",df[col].unique()[0:20])
    print("*****************************************************************************")

column name: Barrier
Number of unique values: 19
Sample values: [ 5  6  7  8  9 10 11  1  0  4  2  3 12 13 14 15 16 17 18]
*****************************************************************************
column name: CourseIndicator
Number of unique values: 4
Sample values: [' ' 'G' 'P' '&']
*****************************************************************************
column name: Distance
Number of unique values: 83
Sample values: [2150. 2575. 2375. 2800. 2400. 2700. 2300. 2850. 1609. 2600. 2950. 2875.
 2650. 2725. 2100. 2675. 2175. 2625. 2000. 2325.]
*****************************************************************************
column name: FoalingCountry
Number of unique values: 15
Sample values: ['FR ' 'ITY' 'SWE' 'NOR' 'FIN' 'DEN' 'SWI' 'BEL' 'GER' 'SPA' 'AUT' 'HOL'
 'USA' 'RUS' 'CZE']
*****************************************************************************
column name: FrontShoes
Number of unique values: 4
Sample values: [0 1 2 3]
************************************************

In [ ]:
# Noon - Drop meaningless column
col_names = ['DamID_Frequency_Flag']
df.drop(columns=col_names, axis=1, inplace=True)

In [ ]:
# flag if the value is 0 or not
df['CourseIndicator_Flag'] = df['CourseIndicator'].apply(lambda x: 0 if x == " " else 1)
df['GoingID'] = df['GoingID'].astype('object')
df['RacingSubType_T'] = df['RacingSubType_T'].replace({True: 1, False: 0})
df['StartType_V'] = df['StartType_V'].replace({True: 1, False: 0})

In [ ]:
for col in df.columns:
    print("column name:",col)
    print("Number of unique values:",df[col].nunique())
    print("Sample values:",df[col].unique()[0:20])
    print("*****************************************************************************")

column name: Barrier
Number of unique values: 19
Sample values: [ 5  6  7  8  9 10 11  1  0  4  2  3 12 13 14 15 16 17 18]
*****************************************************************************
column name: CourseIndicator
Number of unique values: 4
Sample values: [' ' 'G' 'P' '&']
*****************************************************************************
column name: Distance
Number of unique values: 83
Sample values: [2150. 2575. 2375. 2800. 2400. 2700. 2300. 2850. 1609. 2600. 2950. 2875.
 2650. 2725. 2100. 2675. 2175. 2625. 2000. 2325.]
*****************************************************************************
column name: FoalingCountry
Number of unique values: 15
Sample values: ['FR ' 'ITY' 'SWE' 'NOR' 'FIN' 'DEN' 'SWI' 'BEL' 'GER' 'SPA' 'AUT' 'HOL'
 'USA' 'RUS' 'CZE']
*****************************************************************************
column name: FrontShoes
Number of unique values: 4
Sample values: [0 1 2 3]
************************************************

In [ ]:
# Encoding

# frequency encoding for the columns below
frequency_encoding_cols = ['Saddlecloth','CourseIndicator','HorseID','JockeyID','RaceID','SireID','TrackID','TrainerID','NW_Value','FoalingDate_Day_of_Week']

for col in frequency_encoding_cols:
    freq = df[col].value_counts()
    df[col] = df[col].map(freq)

# one-hot encoding for all categorical type columns
df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns)

# min-max scaler to 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

<ipython-input-40-b61d075105c0>:11: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns)


In [ ]:
#df.to_parquet('preprocessed_datatrots_2013-2022.parquet')

In [ ]:
# df.head()
df.columns

Index(['Barrier', 'CourseIndicator', 'Distance', 'FrontShoes',
       'HandicapDistance', 'HindShoes', 'HorseAge', 'HorseID', 'JockeyID',
       'RaceID', 'RacePrizemoney', 'Saddlecloth', 'SireID', 'StartingLine',
       'TrackID', 'TrainerID', 'WeightCarried', 'WetnessScale',
       'AgeRestriction_3', 'AgeRestriction_4', 'AgeRestriction_5',
       'AgeRestriction_6', 'AgeRestriction_7', 'AgeRestriction_8',
       'AgeRestriction_9', 'AgeRestriction_10', 'Barrier_Flag', 'NW_Value',
       'Flag_CA', 'CourseIndicator_Flag', 'DamID_Frequency',
       'FoalingDate_Year', 'FoalingDate_Month', 'FoalingDate_Day_of_Week',
       'win probability', 'HandicapType_Cwt', 'HandicapType_Hcp',
       'HandicapType_SW', 'Race_G1', 'Race_G2', 'Race_G3', 'Race_Evening',
       'Race_Morning', 'RacingSubType_T', 'StartType_V', 'SexRestriction_C&G',
       'SexRestriction_F', 'SexRestriction_M', 'Surface_S', 'Surface_T',
       'FoalingCountry_AUT', 'FoalingCountry_BEL', 'FoalingCountry_CZE',
       'Fo

In [ ]:
threshold = 0.01

# Find columns, the most frequent value's percentage>= 99%
to_drop = [col for col in df.columns if (df[col].value_counts(normalize=True).iloc[0] >= (1 - threshold))]
df = df.drop(columns=to_drop)

In [ ]:
import pandas as pd

df['Race_ID'] = race_id
df['RaceStartTime'] = time
excluded_cols = ['Race_ID', 'RaceStartTime', 'win probability']
new_cols_all = []
new_cols_top7 = []

# Calculate the average values for all players in the same race
for col in tqdm(df.columns.difference(excluded_cols)):
    new_col_all = df.groupby('Race_ID')[col].transform('mean').rename(f'{col}_avg_all')
    new_cols_all.append(new_col_all)

# Calculate the average values for top 7 players in the same race
# mask = df['win probability'] != 0
# for col in tqdm(df.columns.difference(excluded_cols)):
#     new_col_top7 = df[mask].groupby('Race_ID')[col].transform('mean').rename(f'{col}_avg_win_top7').fillna(0)
#     new_cols_top7.append(new_col_top7)

# Concatenate all the new columns to the DataFrame at once
df = pd.concat([df] + new_cols_all, axis=1)


  0%|          | 0/54 [00:00<?, ?it/s]

In [ ]:
df.head()

,Barrier,CourseIndicator,Distance,FrontShoes,HandicapDistance,HindShoes,HorseAge,HorseID,JockeyID,RaceID,...,SexRestriction_M_avg_all,SireID_avg_all,StartType_V_avg_all,StartingLine_avg_all,Surface_S_avg_all,Surface_T_avg_all,TrackID_avg_all,TrainerID_avg_all,WeightCarried_avg_all,WetnessScale_avg_all
0,0.277778,1.0,0.196429,0.0,0.4,0.0,0.307692,0.233480,0.028541,0.470588,...,1.0,0.474558,0.0,0.757576,1.0,0.0,0.223326,0.205228,0.0,0.25
1,0.333333,1.0,0.196429,0.0,0.4,0.0,0.307692,0.070485,0.042098,0.470588,...,1.0,0.474558,0.0,0.757576,1.0,0.0,0.223326,0.205228,0.0,0.25
2,0.388889,1.0,0.196429,0.0,0.4,0.0,0.307692,0.110132,0.057296,0.470588,...,1.0,0.474558,0.0,0.757576,1.0,0.0,0.223326,0.205228,0.0,0.25
3,0.444444,1.0,0.196429,0.0,0.4,0.0,0.307692,0.352423,0.039600,0.470588,...,1.0,0.474558,0.0,0.757576,1.0,0.0,0.223326,0.205228,0.0,0.25
4,0.500000,1.0,0.196429,0.0,0.4,0.0,0.307692,0.202643,0.048091,0.470588,...,1.0,0.474558,0.0,0.757576,1.0,0.0,0.223326,0.205228,0.0,0.25


In [ ]:
df.shape

(1200412, 111)

In [ ]:
df.to_parquet('temp_2013-2022.parquet')
#df = pd.read_parquet(DIR_PATH+'temp_2013-2022.parquet', engine='pyarrow')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DIR_PATH = "/content/drive/MyDrive/CANSSI Competition/"
import pandas as pd
import re
import numpy as np
df = pd.read_parquet(DIR_PATH+'temp_2013-2022.parquet', engine='pyarrow')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


feature selections by correlation

In [ ]:
# correlation_matrix = df.drop(['Race_ID', 'RaceStartTime', 'win probability'], axis=1).corr().abs()
# upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# df_reduced = df.drop(columns=to_drop)
# print(f"Dropped columns: {to_drop}")

In [ ]:
# min-max scaler to 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeric_cols = df.drop(['Race_ID', 'RaceStartTime', 'win probability'], axis=1).select_dtypes(include=['number']).columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [ ]:
time

0         2015-04-18 14:01:00
1         2015-04-18 14:01:00
2         2015-04-18 14:01:00
3         2015-04-18 14:01:00
4         2015-04-18 14:01:00
                  ...        
1200407   2020-01-01 16:30:00
1200408   2020-01-01 16:30:00
1200409   2020-01-01 16:30:00
1200410   2020-01-04 16:28:00
1200411   2020-01-04 16:28:00
Name: RaceStartTime, Length: 1200412, dtype: datetime64[ns]

In [ ]:
df['RaceStartTime'] = time
split_date = pd.Timestamp('2021-11-01')
train = df[df['RaceStartTime'] < split_date]
test = df[df['RaceStartTime'] >= split_date]

In [ ]:
train.head()

,Barrier,CourseIndicator,Distance,FrontShoes,HandicapDistance,HindShoes,HorseAge,HorseID,JockeyID,RaceID,...,GoingAbbrev_H,GoingAbbrev_SO,GoingAbbrev_U,GoingAbbrev_VF,GoingID_1,GoingID_4,GoingID_9,GoingID_12,GoingID_24,RaceStartTime
0,0.277778,1.0,0.196429,0.0,0.4,0.0,0.307692,0.579670,0.021067,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
1,0.333333,1.0,0.196429,0.0,0.4,0.0,0.307692,0.661577,0.119715,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
2,0.388889,1.0,0.196429,0.0,0.4,0.0,0.307692,0.740050,0.107852,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
3,0.444444,1.0,0.196429,0.0,0.4,0.0,0.307692,0.334493,0.112166,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
4,0.500000,1.0,0.196429,0.0,0.4,0.0,0.307692,0.609939,0.040768,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00


In [ ]:
train.shape

(1172292, 78)

In [ ]:
test.shape

(28120, 78)

In [ ]:
train.to_parquet('train_datatrots_2013-2022 (2).parquet')
test.to_parquet('test_datatrots_2013-2022 (2).parquet')

In [ ]:
train = pd.read_parquet(DIR_PATH+'train_datatrots_2013-2022 (2).parquet', engine='pyarrow')
test = pd.read_parquet(DIR_PATH+'test_datatrots_2013-2022 (2).parquet', engine='pyarrow')

In [ ]:
train_ID = train['Race_ID']
test_ID = test['Race_ID']

In [ ]:
X_train = train.drop(columns=['Race_ID','RaceStartTime', 'win probability'])
y_train = train['win probability']
X_test = test.drop(columns=['Race_ID','RaceStartTime', 'win probability'])
y_test = test['win probability']

In [ ]:
# normalize prediction result function
def normalize_predictions_by_id(predictions, ids):
    predictions = np.clip(predictions, 0, 1)
    predictions = pd.Series(predictions)
    ids = pd.Series(ids)
    sums = predictions.groupby(ids).transform('sum').replace(0, pd.NA)
    normalized_predictions = predictions / sums
    normalized_predictions = normalized_predictions.fillna(1.0 / len(predictions))
    return normalized_predictions.values

In [ ]:
y_train_normalized = normalize_predictions_by_id(y_train, train_ID)
y_test_normalized = normalize_predictions_by_id(y_test, test_ID)

In [ ]:
# Xgboost
xgboost = XGBRegressor(random_state=42)
xgboost.fit(X_train, y_train_normalized)
xgb_predictions = xgboost.predict(X_test)

In [ ]:
# Normalize
xgb_predictions = pd.Series(xgb_predictions)
xgb_predictions.index = y_test.index
xgb_pred_norm= normalize_predictions_by_id(xgb_predictions, test_ID)
xgb_mse = mean_squared_error(xgb_pred_norm, y_test_normalized)
print(f'XGBoost MSE: {xgb_mse:.4f}')

XGBoost MSE: 0.0129


In [ ]:
# Lasso Regression
lasso = LassoCV(cv=5).fit(X_train, y_train_normalized)
lasso_predictions = lasso.predict(X_test)
lasso_predictions = pd.Series(lasso_predictions)
lasso_predictions.index = y_test.index
lasso_pred_norm= normalize_predictions_by_id(lasso_predictions, test_ID)
lasso_mse = mean_squared_error(lasso_pred_norm, y_test_normalized)
print(f"Lasso MSE: {lasso_mse:.4f}")

Lasso MSE: 0.0139


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.511e+00, tolerance: 1.912e+00
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Linear Regression
lm = LinearRegression().fit(X_train, y_train_normalized)
lm_predictions = lm.predict(X_test)
lm_predictions = pd.Series(lm_predictions, index=y_test.index)
lm_pred_norm = normalize_predictions_by_id(lm_predictions, test_ID)
lm_mse = mean_squared_error(lm_pred_norm, y_test_normalized)
print(f"Linear Regression MSE: {lm_mse:.4f}")

Linear Regression MSE: 0.0139


In [ ]:
# current best model is XGBoost
xgb_pred_norm

array([0.13504088, 0.13498738, 0.09455411, ..., 0.        , 0.        ,
       0.00422223], dtype=float32)

submission for all observation's forcasting

In [ ]:
X = pd.concat([X_train, X_test], ignore_index=True)
ID = train['Race_ID'].append(test['Race_ID'], ignore_index=True)
y = y_train.append(y_test, ignore_index=True)
y_normalized = normalize_predictions_by_id(y, ID)

<ipython-input-16-837d745f6a55>:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ID = train['Race_ID'].append(test['Race_ID'], ignore_index=True)
<ipython-input-16-837d745f6a55>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y = y_train.append(y_test, ignore_index=True)


In [ ]:
xgb_predictions = xgboost.predict(X)
xgb_predictions = pd.Series(xgb_predictions)
xgb_predictions.index = y.index
xgb_pred_norm= normalize_predictions_by_id(xgb_predictions, ID)
xgb_mse = mean_squared_error(xgb_pred_norm, y_normalized)
print(f'XGBoost MSE: {xgb_mse:.4f}')

XGBoost MSE: 0.0121


In [ ]:
xgb_pred_norm

array([0.15317927, 0.10670125, 0.14966989, ..., 0.        , 0.        ,
       0.00422223], dtype=float32)

In [ ]:
xgb_pred_norm.shape

(1200412,)

In [ ]:
pd.DataFrame({"Win Probability":xgb_pred_norm}).to_parquet('win_probability_variable.parquet')